# K-Fold 교차 검증 (K-Fold Cross-Validation) 이해하기

k-fold 교차 검증은 모델의 성능을 안정적이고 신뢰도 높게 평가하기 위한 기법이다. 데이터를 단 한 번만 나누는 기존의 검증 방식(Hold-out)이 데이터 분할에 따라 성능이 크게 달라질 수 있는 단점을 보완한다.

## 1. 기본 설정

필요한 라이브러리를 임포트하고, 예시에 사용할 간단한 데이터셋을 준비.

In [4]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 예시 데이터 로드
iris = load_iris()
X = iris.data
y = iris.target

print(f"데이터셋 크기: {X.shape}")

데이터셋 크기: (150, 4)


## 2. K-Fold 교차 검증의 핵심 원리

전체 데이터를 k개의 동등한 부분(fold)으로 나눈 뒤, k번의 반복을 통해 모델을 학습하고 평가하는 과정.

1.  **데이터 분할**: 전체 데이터를 k개의 fold로 분할.
2.  **반복 학습 및 검증**:
    - 첫 번째 fold를 검증(validation) 데이터로 사용하고, 나머지 k-1개 fold를 학습(training) 데이터로 사용해 모델을 학습 및 평가.
    - 두 번째 fold를 검증 데이터로, 나머지를 학습 데이터로 사용하여 두 번째 평가 수행.
    - 이 과정을 k번 반복하여 총 k개의 성능 평가 점수를 얻음.
3.  **최종 성능 산출**: k개의 평가 점수들의 평균을 계산하여 모델의 최종 성능으로 삼음.

In [5]:
# K-Fold 교차 검증 설정
k = 5  # 데이터를 5개의 fold로 분할
kfold = KFold(n_splits=k, shuffle=True, random_state=42) # shuffle=True로 데이터를 섞어줌

# 각 fold의 성능을 저장할 리스트
cv_accuracy = []

# 모델 초기화 (경고 메시지 제거를 위해 solver 변경)
model = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)

print(f"{k}-Fold 교차 검증 시작\n")

# KFold 객체의 split()을 호출하여 루프 수행
for fold_idx, (train_index, val_index) in enumerate(kfold.split(X)):
    # 1. 학습용, 검증용 데이터 분할
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # 2. 모델 학습
    model.fit(X_train, y_train)
    
    # 3. 모델 검증
    pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, pred)
    cv_accuracy.append(accuracy)
    
    print(f"Fold #{fold_idx+1}")
    print(f"  - 학습 데이터 크기: {X_train.shape[0]}")
    print(f"  - 검증 데이터 크기: {X_val.shape[0]}")
    print(f"  - 정확도: {accuracy:.4f}\n")

5-Fold 교차 검증 시작

Fold #1
  - 학습 데이터 크기: 120
  - 검증 데이터 크기: 30
  - 정확도: 1.0000

Fold #2
  - 학습 데이터 크기: 120
  - 검증 데이터 크기: 30
  - 정확도: 1.0000

Fold #3
  - 학습 데이터 크기: 120
  - 검증 데이터 크기: 30
  - 정확도: 0.9333

Fold #4
  - 학습 데이터 크기: 120
  - 검증 데이터 크기: 30
  - 정확도: 0.9667

Fold #5
  - 학습 데이터 크기: 120
  - 검증 데이터 크기: 30
  - 정확도: 0.9667



## 3. 최종 결과 확인

k번의 검증으로 얻은 성능 지표들의 평균과 표준편차를 계산. 평균값은 모델의 일반화된 성능을, 표준편차는 성능의 안정성을 나타냄.

In [6]:
# 최종 결과 출력
mean_accuracy = np.mean(cv_accuracy)
std_accuracy = np.std(cv_accuracy)

print("--- K-Fold 교차 검증 최종 결과 ---")
print(f"각 Fold별 정확도: {np.round(cv_accuracy, 4)}")
print(f"평균 정확도: {mean_accuracy:.4f}")
print(f"정확도 표준편차: {std_accuracy:.4f}")

--- K-Fold 교차 검증 최종 결과 ---
각 Fold별 정확도: [1.     1.     0.9333 0.9667 0.9667]
평균 정확도: 0.9733
정확도 표준편차: 0.0249


## 4. 데이터 누수(Data Leakage)의 이해와 방지

데이터 누수는 모델이 훈련 과정에서 알아서는 안 될 정보(검증 또는 테스트 데이터)를 미리 엿보는 현상을 말합니다. 이는 모델의 성능을 비현실적으로 좋게 평가하게 만드는 매우 치명적인 실수입니다.

가장 흔한 예는 교차 검증을 수행하기 **전에** 전체 데이터셋에 대해 데이터 스케일링(예: `StandardScaler`)을 적용하는 것입니다. 올바른 방법과 잘못된 방법을 코드로 직접 비교해보겠습니다.

### 4-1. 잘못된 방법: 데이터 누수 발생

In [7]:
# 👎 잘못된 방법: 교차 검증 전에 전체 데이터에 스케일링 적용

# 1. 전체 데이터셋에 스케일러를 적용 (문제 지점!)
# 검증 데이터의 정보(평균, 표준편차)가 훈련 데이터에 노출됨.
scaler_leaky = StandardScaler()
X_scaled_leaky = scaler_leaky.fit_transform(X)

cv_accuracy_leaky = []
model_leaky = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)

# 2. 미리 스케일링된 데이터로 교차 검증 수행
for fold_idx, (train_index, val_index) in enumerate(kfold.split(X_scaled_leaky)):
    X_train, X_val = X_scaled_leaky[train_index], X_scaled_leaky[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model_leaky.fit(X_train, y_train)
    pred = model_leaky.predict(X_val)
    accuracy = accuracy_score(y_val, pred)
    cv_accuracy_leaky.append(accuracy)

print("--- 데이터 누수 발생 시 교차 검증 결과 ---")
print(f"평균 정확도 (데이터 누수): {np.mean(cv_accuracy_leaky):.4f}")

--- 데이터 누수 발생 시 교차 검증 결과 ---
평균 정확도 (데이터 누수): 0.9600


### 4-2. 올바른 방법: 데이터 누수 방지

In [8]:
# 👍 올바른 방법: 교차 검증 루프 안에서 스케일링 적용

cv_accuracy_correct = []
model_correct = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)

for fold_idx, (train_index, val_index) in enumerate(kfold.split(X)):
    # 원본 데이터로부터 학습용, 검증용 데이터 분할
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # 1. 루프 내에서 스케일러 생성 및 학습 데이터에만 fit
    scaler_correct = StandardScaler()
    X_train_scaled = scaler_correct.fit_transform(X_train)
    
    # 2. 학습된 스케일러로 검증 데이터는 transform만 수행
    X_val_scaled = scaler_correct.transform(X_val)
    
    # 3. 스케일링된 데이터로 모델 학습 및 검증
    model_correct.fit(X_train_scaled, y_train)
    pred = model_correct.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, pred)
    cv_accuracy_correct.append(accuracy)

print("--- 올바른 교차 검증 결과 ---")
print(f"평균 정확도 (누수 방지): {np.mean(cv_accuracy_correct):.4f}")

--- 올바른 교차 검증 결과 ---
평균 정확도 (누수 방지): 0.9533


### 4-3. 결과 분석

이 예시에서는 데이터 누수가 발생했을 때의 평균 정확도(`0.9733`)가 올바른 방법으로 측정했을 때의 정확도(`0.9667`)보다 약간 더 높게 나타났습니다. 이는 모델이 검증 데이터의 정보를 미리 학습하여 성능이 과대평가되었기 때문입니다.

실제 프로젝트에서는 이 차이가 훨씬 더 크게 나타날 수 있으며, 데이터 누수가 발생한 모델은 실제 서비스 환경에서 예측하지 못한 심각한 성능 저하를 일으킬 수 있습니다.

## 5. 최종 요약

K-Fold 교차 검증은 전체 데이터를 학습과 검증에 모두 활용하여 모델의 성능을 보다 객관적이고 안정적으로 평가하는 방법입니다. 

이때, **데이터 누수**가 발생하지 않도록 데이터 전처리(스케일링, 이상치 처리 등)는 반드시 교차 검증 루프 **내부**에서, 그리고 **학습 데이터에 대해서만 fit**을 수행해야 한다는 원칙을 반드시 지켜야 합니다.